In [56]:
# Pkg.add("StatsBase")

In [55]:
using JuMP, Cbc
using Combinatorics
using StatsBase

INFO: Precompiling module StatsBase.


In [75]:
skus = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [76]:
warehouses = [1,2]

2-element Array{Int64,1}:
 1
 2

In [89]:
max_inventory = 3
inventory = rand(1:max_inventory,length(skus),length(warehouses))

3×2 Array{Int64,2}:
 3  1
 3  1
 1  3

In [90]:
inventory[:,1]

3-element Array{Int64,1}:
 3
 3
 1

In [96]:
min_order_size = 1
max_order_size = length(skus)

3

In [97]:
# sku_combos = [collect(combinations(skus, max_order_size))]

In [98]:
sku_combos = [collect(combinations(skus, o)) for o in min_order_size:max_order_size];

In [99]:
orders = []
for i in 1:length(sku_combos)
    append!(orders, sku_combos[i])
end

In [100]:
orders

7-element Array{Any,1}:
 [1]      
 [2]      
 [3]      
 [1, 2]   
 [1, 3]   
 [2, 3]   
 [1, 2, 3]

In [123]:
wh_1_allocation = .3
orders_wh_1 = sample(orders, Int(round(length(orders) * wh_1_allocation, 0)), replace=false)
orders_wh = [[o, o in wc ? 1 : 2] for o in orders]

7-element Array{Array{Any,1},1}:
 Any[[1], 1]      
 Any[[2], 2]      
 Any[[3], 1]      
 Any[[1, 2], 1]   
 Any[[1, 3], 1]   
 Any[[2, 3], 2]   
 Any[[1, 2, 3], 1]

In [107]:
A = [Int(D[i, j] <= max_miles) for i=1:m, j=1:m]

5×5 Array{Int64,2}:
 1  1  0  0  0
 1  1  0  0  1
 0  0  1  1  0
 0  0  1  1  1
 0  1  0  1  1

In [124]:
model = Model(solver=CbcSolver())

# decision variable (binary): whether to build warehouse near distribution center i
@variable(model, y[1:m], Bin)

# Objective: minimize number of warehouses
@objective(model, Min, sum(y))

# Constraint: has to cover all warehouses
# (.>= is the element-wise dot comparison operator)
@constraint(model, A*y .>= 1)

model

Minimization problem with:
 * 5 linear constraints
 * 5 variables: 5 binary
Solver is CbcMathProg

We have an additional constraint that at least 1 warehouse should be within 10 miles of distribution center 1, but our activity matrix $A$ already covers that, so technically we do not need this explicit constraint.

In [125]:
@constraint(model, y[1] + y[2] >= 1)

y[1] + y[2] ≥ 1

In [126]:
# Solve problem using MIP solver
status = solve(model)

:Optimal

In [137]:
println("Total # of warehouses: ", getobjectivevalue(model))

println("Build warehouses at distribution center(s):")

[i for i=1:m if getvalue(y[i]) == 1 ]

Total # of warehouses: 2.0
Build warehouses at distribution center(s):


2-element Array{Int64,1}:
 2
 3